In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize TPU strategy
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

# Define hyperparameters
batch_size = 32 * strategy.num_replicas_in_sync
epochs = 20
image_size = (128, 128)
num_classes = 2  # Binary classification (cats and dogs)

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/cat-and-dog/training_set',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Binary classification
)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/cat-and-dog/test_set',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Binary classification
)

# Build the CNN model
with strategy.scope():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',  # Binary cross-entropy
        metrics=['accuracy']
    )

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=epochs,
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

# Save the model
model.save('cats_vs_dogs_model.h5')


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


free(): corrupted unsorted chunks
https://symbolize.stripped_domain/r/?trace=7895bc11eccc,7895bc0cff8f,5944d617c14f&map= 
*** SIGABRT received by PID 3344 (TID 4262) on cpu 31 from PID 3344; stack trace: ***
PC: @     0x7895bc11eccc  (unknown)  (unknown)
    @     0x789572c3605a       1152  (unknown)
    @     0x7895bc0cff90      22608  (unknown)
    @     0x5944d617c150  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7895bc11eccc,789572c36059,7895bc0cff8f,5944d617c14f&map=1278088d049ad36cb636fbbc76303cb3:789567600000-789572e4d7c0 
E0903 08:35:29.020737    4262 coredump_hook.cc:414] RAW: Remote crash data gathering hook invoked.
E0903 08:35:29.020758    4262 client.cc:278] RAW: Coroner client retries enabled (b/136286901), will retry for up to 30 sec.
E0903 08:35:29.020762    4262 coredump_hook.cc:512] RAW: Sending fingerprint to remote end.
E0903 08:35:29.020769    4262 coredump_socket.cc:120] RAW: Stat failed errno=2 on socket /var/google/services/logmanagerd/remote_

In [1]:
import tensorflow as tf

# Check if TPU is available
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    print("TPU is available and initialized.")
    print(f"TPU device: {tpu.master()}")
except Exception as e:
    print("No TPU available.")


D0903 08:35:08.868172888    3344 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0903 08:35:08.868204397    3344 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0903 08:35:08.868207797    3344 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0903 08:35:08.868210453    3344 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0903 08:35:08.868212834    3344 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0903 08:35:08.868215106    3344 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0903 08:35:08.868217663    3344 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0903 08:35:08.

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
TPU is available and initialized.
TPU device: 


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Check if a GPU is available
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Allow GPU memory growth, which prevents allocation errors
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is available and configured.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU available. Switching to CPU.")

# Define hyperparameters
batch_size = 32
epochs = 20
image_size = (128, 128)

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load and preprocess your dataset using the provided generators
train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/cat-and-dog/training_set',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Binary classification
)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/cat-and-dog/test_set',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Binary classification
)

# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Binary cross-entropy
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=epochs,
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

# Save the model
model.save('cats_vs_dogs_model.h5')


GPU is available and configured.
Found 8005 images belonging to 1 classes.
Found 2023 images belonging to 1 classes.
Epoch 1/20
251/251 [==============================] - 120s 444ms/step - loss: 0.0025 - accuracy: 0.9996 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/20
251/251 [==============================] - 49s 194ms/step - loss: 5.8084e-36 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/20
251/251 [==============================] - 48s 191ms/step - loss: 1.2582e-38 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/20
251/251 [==============================] - 49s 195ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/20
251/251 [==============================] - 48s 192ms/step - loss: 6.9840e-38 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
251/251 [==============================] - 49s 197ms/step - loss: 5.0102e-33 - accuracy: 1.0000 - val_loss: 0.